<a href="https://www.kaggle.com/code/averma111/ps3e13-map-k-tensorflow?scriptVersionId=126743685" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [162]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e13/sample_submission.csv
/kaggle/input/playground-series-s3e13/train.csv
/kaggle/input/playground-series-s3e13/test.csv


## Reading the train data using pandas


In [163]:
ROOT_PATH ='/kaggle/input/playground-series-s3e13'
train = pd.read_csv(ROOT_PATH+'/train.csv')
train.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


## Check the summary of the train dataset

In [164]:
def dataset_summary(df):
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary['Null'] = df.isnull().sum()
    summary['Unique'] = df.nunique()
    summary['min'] = df.min()
    summary['median'] = df.median()
    summary['max'] = df.max()
    summary['mean'] = df.mean()
    summary['std'] = df.std()
    
    return summary

In [165]:
dataset_summary(train)

,dtypes,Null,Unique,min,median,max,mean,std
id,int64,0,707,0,353.0,706,353.000000,204.237607
sudden_fever,float64,0,2,0.0,1.0,1.0,0.503536,0.500341
headache,float64,0,2,0.0,0.0,1.0,0.449788,0.497825
mouth_bleed,float64,0,2,0.0,0.0,1.0,0.459689,0.498725
nose_bleed,float64,0,2,0.0,0.0,1.0,0.487977,0.500209
...,...,...,...,...,...,...,...,...
ulcers,float64,0,2,0.0,0.0,1.0,0.144272,0.351614
toenail_loss,float64,0,2,0.0,0.0,1.0,0.137199,0.344301
speech_problem,float64,0,2,0.0,0.0,1.0,0.032532,0.177533
bullseye_rash,float64,0,2,0.0,0.0,1.0,0.031117,0.173758


## Observation from summary of train dataset
* There are no null value in any columns
* Apart from Id and prognosis all the columns are categorial
* The categorical datatype is float64

In [166]:
from sklearn.decomposition import PCA
def do_pca(df):
    df = df.drop('id',axis = 1)
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    pca = PCA(n_components = 5, random_state = 42)
    X = pd.DataFrame(pca.fit_transform(X))
    X['target'] = y
    return X

In [167]:
train = do_pca(train)

In [168]:
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.utils import to_categorical

# Create X and y
X = train.iloc[:,:-1]
y = train[['target']]

enc = OrdinalEncoder()
y = to_categorical(enc.fit_transform(y))
N_CLASSES = len(enc.categories_[0])
                
# Split between train and test dataset
X_train,X_test,y_train,y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=42)

## Defining the MAP@K function


In [169]:
import numpy as np
def apk(actual, predicted, k=3):
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3): 
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Basic_Model

In [172]:
def better_model(INPUT_SHAPE,
                FIRST_NODE,
                LAYERS,
                HIDDEN_NODE,
                L2,
                DROPOUT_RATE,
                OUTPUT_SHAPE):
    
    inputs = tf.keras.layers.Input(shape=INPUT_SHAPE)
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(FIRST_NODE, activation='relu')(x)
    for _ in range(LAYERS):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(HIDDEN_NODE,kernel_regularizer = tf.keras.regularizers.l2(L2),
                         activation='relu')(x)
        x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
        
    outputs = tf.keras.layers.Dense(OUTPUT_SHAPE, activation = 'softmax')(x)
    return tf.keras.Model(inputs,outputs)

In [173]:
params = {'INPUT_SHAPE':X_train.shape[1:],
            'FIRST_NODE':64,
            'LAYERS':3,
            'HIDDEN_NODE':32,
            'L2': 0.01,
            'DROPOUT_RATE':0.3,
            'OUTPUT_SHAPE':N_CLASSES}

optimizers = {'loss': tf.keras.losses.CategoricalCrossentropy(),
              'optimizer': tf.keras.optimizers.legacy.Adam(learning_rate = 0.01),
              'metrics':['accuracy']}

model = better_model(**params)
model.compile(**optimizers)

In [174]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)

In [175]:
history = model.fit(X_train,y_train,
                   validation_data = (X_test,y_test),
                   epochs = 9999,
                   callbacks = [early_stopping],verbose=0)

ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1083, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
 

In [ ]:
def make_plot(history):
    
    fig,ax = plt.subplots(ncols = 2, figsize = (15,5))
    
    ax[0].plot(history.history['loss'], label = 'train_loss', c = 'orange')
    ax[0].plot(history.history['val_loss'], label = 'valid_loss', c = 'red')
    ax[0].legend(loc = 'upper right')
    ax[0].set_title('train,valid loss')
    
    ax[1].plot(history.history['accuracy'], label = 'train_accuracy', c = 'orange')
    ax[1].plot(history.history['val_accuracy'], label = 'valid_accuracy', c = 'red')
    ax[1].legend(loc = 'lower right')
    ax[1].set_title('train,valid accuracy')

In [ ]:
make_plot(history)
plt.suptitle(f'MPK : {mpk(y_test,better_model.predict(x_test)):.3f}')

In [ ]:
pca = PCA(n_components = 5, random_state = 42)

test = pca.fit_transform(test.drop('id',axis = 1))

In [176]:
def make_sub(probs):
    result = []
    for prob in probs:
        result.append(prob.argsort()[-3:][::-1])
    reverse = {}
    for i,cate in enumerate(enc.categories_[0]):
        reverse[i] = cate
    df = pd.DataFrame(result)
    to_sub = df[0].map(reverse) + ' ' + df[1].map(reverse) + ' ' + df[2].map(reverse)
    return to_sub

In [ ]:
sub['prognosis'] = make_sub(model.predict(test))